In [ ]:
!unzip '/content/drive/My Drive/NNProjectk.zip'

Streaming output truncated to the last 5000 lines.
  inflating: NNProject/train/n04067472/images/n04067472_73.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_72.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_71.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_70.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_7.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_69.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_68.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_67.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_66.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_65.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_64.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_63.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_62.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_61.JPEG  
  inflating: NNProject/tra

In [ ]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf
tf.test.gpu_device_name()
from keras.preprocessing.image import ImageDataGenerator
import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from keras.models import Model
import pandas as pd
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D,Flatten
from keras.layers.normalization import BatchNormalization   

#creating the inception layer
def inception_module(input_img,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj):
    tower_1 = Conv2D(filters_1x1, (1,1), padding='same', activation='relu')(input_img)
   
    tower_2 = Conv2D(filters_3x3_reduce, (1,1), padding='same', activation='relu')(input_img)
    tower_3 = Conv2D(filters_3x3, (2,2), padding='same', activation='relu')(tower_2)
   
    tower_4 = Conv2D(filters_5x5_reduce, (1,1), padding='same', activation='relu')(input_img)    
    tower_5 = Conv2D(filters_5x5, (3,3), padding='same', activation='relu')(tower_4)
    tower_5=  BatchNormalization()(tower_5)
    tower_6 = MaxPool2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_7 = Conv2D(filters_pool_proj, (1,1), padding='same', activation='relu')(tower_6)
   
    output = keras.layers.concatenate([tower_1, tower_3, tower_5, tower_7], axis = 3)
    return output
    
#get multiple outputs for train data        
def multiple_outputs(generator,valData, image_dir, batch_size, image_size, subset):
        gen = generator.flow_from_directory(
        image_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        subset=subset)   
    while True:
        gnext = gen.next()
        # return image batch and 3 sets of lables 
        #required 3 sets as we have two auxiliary classfiers and we have to add 3 labels in model.fit
        yield gnext[0], [gnext[1], gnext[1], gnext[1]]

#multiple outputs for validation dataset        
def get_val(gen):
     while True:
        gnext = gen.next()
        # return image batch and 3 sets of lables
        yield gnext[0], [gnext[1], gnext[1], gnext[1]]
    
image_size= 224
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip =True,
    )

train_generator = multiple_outputs(
    train_datagen, valData='',
    image_dir="NNProject/train/",
    batch_size=32,
    image_size=image_size,
    subset='training')

val_datagen = ImageDataGenerator(rescale=1./255)
val_data = pd.read_csv('NNProject/val/val_annotations.txt', sep='\t',header=None,names=['File','Class','X','Y','H','W'])
val_data.drop(['X','Y','H','W'], axis=1, inplace=True)
val_data.head()


val = val_datagen.flow_from_dataframe(val_data,"NNProject/val/images", 
                                      x_col='File', y_col ='Class' ,
                                      class_mode="categorical",
                                      batch_size=40,
                                      target_size=(224, 224))

validation_generator = get_val(val)      

input_img = Input(shape = (224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input_img)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((2, 2), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

# 3 a
x =inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32)
x=  BatchNormalization()(x)
#3b
x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64
                     )

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)
x=  BatchNormalization()(x)
#4a
x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64
                     )

x1 = AveragePooling2D((3, 3), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1=  BatchNormalization()(x1)
x1 = Flatten()(x1)
x1 = Dense(4096, activation='relu')(x1)

x1 = Dropout(0.5)(x1)
x1 = Dense(200, activation='softmax', name='auxilliary_output_1')(x1)

## inception 4b
x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64
                    )
x=  BatchNormalization()(x)
#inception 4c
x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64
                    )
x=  BatchNormalization()(x)
#name='inception_4d'
x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64
                     )

x2 = AveragePooling2D((3, 3), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2=  BatchNormalization()(x2)
x2 = Flatten()(x2)
x2 = Dense(4096, activation='relu')(x2)
x2 = Dropout(0.5)(x2)
x2 = Dense(200, activation='softmax', name='auxilliary_output_2')(x2)

# name='inception_4e'
x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128
                    )

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)
#name='inception_5a'
x=  BatchNormalization()(x)
x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128
                     )
#name='inception_5b'
x=  BatchNormalization()(x)
x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128
                     )
x=  BatchNormalization()(x)
x= GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x= Dropout(0.4)(x)

x= Dense(200, activation='softmax', name='output')(x)


model = Model(inputs= input_img,outputs= [x,x1,x2])

model.summary()
epochs = 100
lrate = 0.001
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

model.compile( optimizer=sgd,
              loss = ["categorical_crossentropy","categorical_crossentropy","categorical_crossentropy"],
    metrics = ["accuracy"])

lr_sc = LearningRateScheduler(decay, verbose=1)

model.fit_generator(
    train_generator,
    steps_per_epoch = 2812, # batch_size,
    validation_data = validation_generator, 
    validation_steps = 312, # batch_size,
    epochs = 100)

Using TensorFlow backend.


Found 10000 validated image filenames belonging to 200 classes.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 56, 56, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 56, 56, 64)   4160        max_pool_1_3x3/2[0][0]           
____________________________

**Results obtained from the model coded from scratch were not as high as we can obtain from Transfer learning on Inception Model of keras/Pytorch**